# Bonus: CNN zur Erkennung von Personen und Autos

Dieses Notebook implementiert ein CNN zur Erkennung von Personen und Autos und wendet es auf Bilder an, die sowohl Personen als auch Autos enthalten.

## Überblick
- Laden und Vorbereiten des Datensatzes für Personenerkennung
- Training eines CNN-Modells für Personenerkennung
- Laden des vortrainierten Autoerkennungsmodells
- Implementierung einer verbesserten Erkennungsmethode basierend auf dem 05-Notebook
- Anwendung auf Testbilder mit Personen und Autos
- Visualisierung der Ergebnisse

## Importieren der benötigten Bibliotheken

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import requests
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont
import time
import glob
import random
from sklearn.model_selection import train_test_split

## Vorbereitung der Verzeichnisse

Wir erstellen die notwendigen Verzeichnisse für Modelle, Bilder, Ergebnisse und Bonus-Ergebnisse.

In [ ]:
# Verzeichnisse
models_dir = '../models'
data_dir = '../data'
images_dir = '../images'
results_dir = '../results'
bonus_dir = '../bonus'
human_dataset_dir = '/home/ubuntu/HumanBinaryClassificationSuite/dataset'

os.makedirs(models_dir, exist_ok=True)
os.makedirs(os.path.join(models_dir, 'human_detection'), exist_ok=True)
os.makedirs(images_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(bonus_dir, exist_ok=True)

## Laden und Vorbereiten des Datensatzes für Personenerkennung

Wir laden den Datensatz aus dem HumanBinaryClassificationSuite Repository und bereiten ihn für das Training vor.

In [ ]:
def load_and_prepare_dataset(dataset_dir, target_size=(32, 32), batch_size=32, validation_split=0.2):
    """
    Lädt den Datensatz und bereitet ihn für das Training vor.
    
    Args:
        dataset_dir: Verzeichnis mit den Unterordnern 'human' und 'nonhuman'
        target_size: Zielgröße für die Bilder
        batch_size: Batch-Größe für das Training
        validation_split: Anteil der Daten für die Validierung
        
    Returns:
        train_generator: Generator für Trainingsdaten
        validation_generator: Generator für Validierungsdaten
        class_weights: Gewichtung der Klassen für unbalancierte Daten
    """
    print(f"Lade Datensatz aus {dataset_dir}...")
    
    # Datenaugmentation für Trainingsdaten
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=validation_split
    )
    
    # Generator für Trainingsdaten
    train_generator = train_datagen.flow_from_directory(
        dataset_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='training'
    )
    
    # Generator für Validierungsdaten
    validation_generator = train_datagen.flow_from_directory(
        dataset_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation'
    )
    
    # Berechnen der Klassengewichte für unbalancierte Daten
    human_count = len(os.listdir(os.path.join(dataset_dir, 'human')))
    nonhuman_count = len(os.listdir(os.path.join(dataset_dir, 'nonhuman')))
    total_count = human_count + nonhuman_count
    
    print(f"Anzahl der Bilder: Human = {human_count}, Nonhuman = {nonhuman_count}, Gesamt = {total_count}")
    
    # Klassengewichte berechnen (höheres Gewicht für die unterrepräsentierte Klasse)
    weight_for_0 = (1 / nonhuman_count) * (total_count / 2.0)  # nonhuman
    weight_for_1 = (1 / human_count) * (total_count / 2.0)     # human
    
    class_weights = {0: weight_for_0, 1: weight_for_1}
    print(f"Klassengewichte: {class_weights}")
    
    return train_generator, validation_generator, class_weights

In [ ]:
# Laden und Vorbereiten des Datensatzes
train_generator, validation_generator, class_weights = load_and_prepare_dataset(
    human_dataset_dir,
    target_size=(32, 32),
    batch_size=64,
    validation_split=0.2
)

## Definition des CNN-Modells für Personenerkennung

Wir definieren ein CNN-Modell zur Erkennung von Personen. Das Modell besteht aus mehreren Convolutional Blocks mit Batch Normalization und Dropout zur Regularisierung.

In [ ]:
def create_person_detection_model(input_shape=(32, 32, 3)):
    model = Sequential([
        # Erster Convolutional Block
        Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(32, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Zweiter Convolutional Block
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Dritter Convolutional Block
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Fully Connected Layers
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binäre Klassifikation: Person vs. Nicht-Person
    ])
    
    # Kompilieren des Modells
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

## Training des Personenerkennungsmodells

Wir trainieren das Modell mit dem vorbereiteten Datensatz und speichern das beste Modell.

In [ ]:
# Erstellen des Modells
person_model = create_person_detection_model(input_shape=(32, 32, 3))

# Zusammenfassung des Modells anzeigen
person_model.summary()

In [ ]:
# Callbacks für das Training
checkpoint_path = os.path.join(models_dir, 'human_detection', 'human_detection_model.keras')
checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Training des Modells
print("Training des Personenerkennungsmodells...")
history = person_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=30,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint, early_stopping],
    class_weight=class_weights,
    verbose=1
)

## Visualisierung des Trainingsverlaufs

In [ ]:
# Visualisierung des Trainingsverlaufs
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validierung')
plt.title('Genauigkeit')
plt.xlabel('Epoch')
plt.ylabel('Genauigkeit')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validierung')
plt.title('Verlust')
plt.xlabel('Epoch')
plt.ylabel('Verlust')
plt.legend()

plt.tight_layout()
plt.show()

## Laden der trainierten Modelle

Wir laden das trainierte Personenerkennungsmodell und das vorhandene Autoerkennungsmodell.

In [ ]:
# Laden des trainierten Personenerkennungsmodells
print("Laden des trainierten Personenerkennungsmodells...")
try:
    person_model = load_model(os.path.join(models_dir, 'human_detection', 'human_detection_model.keras'))
    print("Personenerkennungsmodell erfolgreich geladen.")
except:
    print("Fehler beim Laden des Personenerkennungsmodells. Verwende das zuletzt trainierte Modell.")

# Laden des trainierten Auto-Modells
print("Laden des trainierten Auto-Modells...")
try:
    car_model = load_model(os.path.join(models_dir, 'keras_cnn', 'car_detection_model.keras'))
    print("Auto-Modell erfolgreich geladen.")
except:
    print("Fehler beim Laden des Auto-Modells. Bitte stellen Sie sicher, dass das Modell trainiert wurde.")
    raise Exception("Auto-Modell konnte nicht geladen werden.")

## Funktionen für die Bildverarbeitung und Objekterkennung

### Laden und Vorverarbeiten von Bildern

In [ ]:
def load_and_preprocess_image(image_path, target_size=(32, 32)):
    """
    Lädt ein Bild und bereitet es für die Vorhersage vor.
    
    Args:
        image_path: Pfad zum Bild oder URL
        target_size: Zielgröße für das Modell
        
    Returns:
        image: Originalbild
        processed_image: Vorverarbeitetes Bild für das Modell
        (original_height, original_width): Originalgröße des Bildes
    """
    # Überprüfen, ob es sich um eine URL handelt
    if image_path.startswith('http'):
        response = requests.get(image_path)
        pil_image = Image.open(BytesIO(response.content))
    else:
        pil_image = Image.open(image_path)
    
    # Convert PIL image to numpy array
    image = np.array(pil_image)
    
    # Speichern der Originalgröße
    original_height, original_width = image.shape[:2]
    
    # Vorverarbeitung für das Modell
    pil_resized = pil_image.resize(target_size)
    processed_image = np.array(pil_resized).astype('float32') / 255.0
    
    return image, processed_image, (original_height, original_width)

### Region Proposals für Objekterkennung

Wir implementieren eine Methode für Region Proposals, die auf dem 05-Notebook basiert.

In [ ]:
def generate_region_proposals(image, method='fast'):
    """
    Generiert Regionsvorschläge für die Objekterkennung.
    
    Args:
        image: Eingabebild
        method: 'fast' oder 'quality' für die Anzahl der generierten Regionen
        
    Returns:
        regions: Liste der vorgeschlagenen Regionen (x, y, w, h)
    """
    height, width = image.shape[:2]
    regions = []
    
    # Verschiedene Fenstergrößen und Schrittweiten basierend auf der Methode
    if method == 'fast':
        window_sizes = [(64, 64), (96, 96), (128, 128), (196, 196), (256, 256)]
        strides = [32, 48, 64, 98, 128]
    else:  # 'quality'
        window_sizes = [(64, 64), (96, 96), (128, 128), (160, 160), (196, 196), (224, 224), (256, 256), (320, 320)]
        strides = [16, 24, 32, 48, 64, 80, 96, 128]
    
    # Generieren von Regionen mit verschiedenen Fenstergrößen und Schrittweiten
    for window_size, stride in zip(window_sizes, strides):
        for y in range(0, height - window_size[1], stride):
            for x in range(0, width - window_size[0], stride):
                regions.append((x, y, window_size[0], window_size[1]))
    
    # Zusätzliche Regionen für verschiedene Seitenverhältnisse
    aspect_ratios = [0.5, 0.75, 1.0, 1.5, 2.0]
    base_sizes = [64, 96, 128, 196, 256]
    
    for base_size in base_sizes:
        for ratio in aspect_ratios:
            w = int(base_size * ratio)
            h = int(base_size / ratio)
            stride = base_size // 2
            
            if w <= width and h <= height:
                for y in range(0, height - h, stride):
                    for x in range(0, width - w, stride):
                        regions.append((x, y, w, h))
    
    # Filtern der Regionen nach Größe
    min_area = 500  # Minimale Fläche für eine Region
    max_area = image.shape[0] * image.shape[1] * 0.8  # Maximale Fläche (80% des Bildes)
    
    filtered_regions = []
    for x, y, w, h in regions:
        area = w * h
        if min_area <= area <= max_area and w/h >= 0.5 and w/h <= 2.0:  # Verhältnis von Breite zu Höhe
            filtered_regions.append((x, y, w, h))
    
    # Entfernen von Duplikaten
    filtered_regions = list(set(filtered_regions))
    
    # Begrenzen der Anzahl der Regionen, um die Verarbeitung zu beschleunigen
    max_regions = 300 if method == 'fast' else 500
    if len(filtered_regions) > max_regions:
        # Sortieren nach Größe (absteigend) und Auswahl der größten Regionen
        filtered_regions.sort(key=lambda r: r[2] * r[3], reverse=True)
        filtered_regions = filtered_regions[:max_regions]
    
    return filtered_regions

### Objekterkennung mit Region Proposals

Diese Funktionen erkennen Objekte in einem Bild mit Region Proposals.

In [ ]:
def detect_objects_with_region_proposals(image, model, object_type, confidence_threshold=0.7):
    """
    Erkennt Objekte in einem Bild mit Region Proposals.
    
    Args:
        image: Eingabebild
        model: Trainiertes Modell
        object_type: 'person' oder 'car'
        confidence_threshold: Schwellenwert für die Konfidenz
        
    Returns:
        detections: Liste der erkannten Objekte (x, y, w, h, confidence, object_type)
    """
    # Regionen mit Region Proposals vorschlagen
    regions = generate_region_proposals(image)
    
    detections = []
    
    for x, y, w, h in regions:
        # Extrahieren der Region
        if y + h <= image.shape[0] and x + w <= image.shape[1]:  # Sicherstellen, dass die Region innerhalb des Bildes liegt
            region = image[y:y+h, x:x+w]
            
            # Vorverarbeitung der Region
            try:
                pil_region = Image.fromarray(region)
                region_resized = np.array(pil_region.resize((32, 32)))
                region_normalized = region_resized.astype('float32') / 255.0
                region_batch = np.expand_dims(region_normalized, axis=0)
                
                # Vorhersage
                prediction = model.predict(region_batch, verbose=0)[0][0]
                
                # Wenn die Konfidenz über dem Schwellenwert liegt, speichern wir die Erkennung
                if prediction > confidence_threshold:
                    detections.append((x, y, w, h, prediction, object_type))
            except Exception as e:
                # Ignorieren von Regionen, die nicht verarbeitet werden können
                continue
    
    return detections

In [ ]:
def detect_objects_multi_scale(image, model, object_type, scales=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5], 
                              confidence_threshold=0.7):
    """
    Erkennt Objekte in einem Bild mit Multi-Scale-Erkennung.
    
    Args:
        image: Eingabebild
        model: Trainiertes Modell
        object_type: 'person' oder 'car'
        scales: Liste der Skalierungsfaktoren
        confidence_threshold: Schwellenwert für die Konfidenz
        
    Returns:
        detections: Liste der erkannten Objekte (x, y, w, h, confidence, object_type)
    """
    height, width = image.shape[:2]
    detections = []
    
    # Erkennung mit Region Proposals auf dem Originalbild
    detections.extend(detect_objects_with_region_proposals(image, model, object_type, confidence_threshold))
    
    # Multi-Scale-Erkennung
    for scale in scales:
        # Skalieren des Bildes
        scaled_height = int(height * scale)
        scaled_width = int(width * scale)
        
        # Verwenden von PIL für die Skalierung
        pil_image = Image.fromarray(image)
        scaled_image = np.array(pil_image.resize((scaled_width, scaled_height)))
        
        # Erkennen von Objekten im skalierten Bild mit Region Proposals
        scaled_detections = detect_objects_with_region_proposals(scaled_image, model, object_type, confidence_threshold)
        
        # Anpassen der Koordinaten an die Originalgröße
        for (x, y, w, h, conf, obj_type) in scaled_detections:
            x_orig = int(x / scale)
            y_orig = int(y / scale)
            w_orig = int(w / scale)
            h_orig = int(h / scale)
            detections.append((x_orig, y_orig, w_orig, h_orig, conf, obj_type))
    
    return detections

In [ ]:
def detect_large_objects(image, model, object_type, confidence_threshold=0.6):
    """
    Erkennt große Objekte, die einen signifikanten Teil des Bildes einnehmen.
    
    Args:
        image: Eingabebild
        model: Trainiertes Modell
        object_type: 'person' oder 'car'
        confidence_threshold: Schwellenwert für die Konfidenz
        
    Returns:
        detections: Liste der erkannten großen Objekte (x, y, w, h, confidence, object_type)
    """
    height, width = image.shape[:2]
    detections = []
    
    # Definieren von großen Regionen, die signifikante Teile des Bildes abdecken
    # Wir erstellen Regionen, die 40% bis 90% des Bildes abdecken
    coverage_ratios = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    for ratio in coverage_ratios:
        # Berechnen der Größe der Region basierend auf dem Verhältnis
        region_width = int(width * ratio)
        region_height = int(height * ratio)
        
        # Berechnen der Position der Region (zentriert)
        x = (width - region_width) // 2
        y = (height - region_height) // 2
        
        # Extrahieren der Region
        region = image[y:y+region_height, x:x+region_width]
        
        # Vorverarbeitung der Region
        try:
            pil_region = Image.fromarray(region)
            region_resized = np.array(pil_region.resize((32, 32)))
            region_normalized = region_resized.astype('float32') / 255.0
            region_batch = np.expand_dims(region_normalized, axis=0)
            
            # Vorhersage
            prediction = model.predict(region_batch, verbose=0)[0][0]
            
            # Wenn die Konfidenz über dem Schwellenwert liegt, speichern wir die Erkennung
            if prediction > confidence_threshold:
                detections.append((x, y, region_width, region_height, prediction, object_type))
        except Exception as e:
            # Ignorieren von Regionen, die nicht verarbeitet werden können
            continue
    
    return detections

### Erweiterte Non-Maximum Suppression

Diese Funktion implementiert eine erweiterte Non-Maximum Suppression, die 100% überlappende Boxen entfernt und eine intelligente Filterung für verschachtelte Boxen bietet.

In [ ]:
def advanced_non_max_suppression(boxes, overlap_threshold=0.3, score_threshold=0.6, containment_threshold=0.95):
    """
    Führt eine erweiterte Non-Maximum Suppression durch, die 100% überlappende Boxen entfernt
    und eine intelligente Filterung für verschachtelte Boxen implementiert.
    
    Args:
        boxes: Liste der Bounding Boxes (x, y, w, h, confidence, object_type)
        overlap_threshold: Schwellenwert für die Überlappung (IoU)
        score_threshold: Minimaler Konfidenzwert
        containment_threshold: Schwellenwert für die Enthaltung einer Box in einer anderen
        
    Returns:
        picked: Liste der ausgewählten Bounding Boxes
    """
    if len(boxes) == 0:
        return []
    
    # Filtern nach Konfidenz
    boxes = [box for box in boxes if box[4] >= score_threshold]
    
    if len(boxes) == 0:
        return []
    
    # Gruppieren der Boxen nach Objekttyp
    person_boxes = [box for box in boxes if box[5] == 'person']
    car_boxes = [box for box in boxes if box[5] == 'car']
    
    # Verarbeiten der Boxen für jeden Objekttyp separat
    picked_persons = process_boxes_by_type(person_boxes, overlap_threshold, containment_threshold)
    picked_cars = process_boxes_by_type(car_boxes, overlap_threshold, containment_threshold)
    
    # Kombinieren der Ergebnisse
    return picked_persons + picked_cars

def process_boxes_by_type(boxes, overlap_threshold, containment_threshold):
    """
    Verarbeitet Bounding Boxes eines bestimmten Typs mit Non-Maximum Suppression.
    
    Args:
        boxes: Liste der Bounding Boxes (x, y, w, h, confidence, object_type)
        overlap_threshold: Schwellenwert für die Überlappung (IoU)
        containment_threshold: Schwellenwert für die Enthaltung einer Box in einer anderen
        
    Returns:
        picked: Liste der ausgewählten Bounding Boxes
    """
    if len(boxes) == 0:
        return []
    
    # Konvertieren der Bounding Boxes in das Format (x1, y1, x2, y2, conf, object_type)
    boxes_array = np.array([(x, y, x + w, y + h, conf, obj_type) for x, y, w, h, conf, obj_type in boxes])
    
    # Sortieren der Bounding Boxes nach Konfidenz (absteigend)
    boxes_array = boxes_array[np.argsort(boxes_array[:, 4])[::-1]]
    
    # Entfernen von 100% überlappenden Boxen (Duplikate)
    unique_boxes = []
    for box in boxes_array:
        is_duplicate = False
        for unique_box in unique_boxes:
            if (box[0] == unique_box[0] and box[1] == unique_box[1] and 
                box[2] == unique_box[2] and box[3] == unique_box[3]):
                is_duplicate = True
                break
        if not is_duplicate:
            unique_boxes.append(box)
    
    boxes_array = np.array(unique_boxes)
    
    picked = []
    
    while len(boxes_array) > 0:
        # Die Box mit der höchsten Konfidenz auswählen
        current_box = boxes_array[0]
        picked.append(current_box)
        
        # Berechnen der Überlappung mit den verbleibenden Boxen
        remaining_boxes = boxes_array[1:]
        
        if len(remaining_boxes) == 0:
            break
        
        # Berechnen der Koordinaten der Überlappung
        xx1 = np.maximum(current_box[0], remaining_boxes[:, 0])
        yy1 = np.maximum(current_box[1], remaining_boxes[:, 1])
        xx2 = np.minimum(current_box[2], remaining_boxes[:, 2])
        yy2 = np.minimum(current_box[3], remaining_boxes[:, 3])
        
        # Berechnen der Breite und Höhe der Überlappung
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        
        # Berechnen des Überlappungsverhältnisses (IoU - Intersection over Union)
        intersection = w * h
        area1 = (current_box[2] - current_box[0] + 1) * (current_box[3] - current_box[1] + 1)
        area2 = (remaining_boxes[:, 2] - remaining_boxes[:, 0] + 1) * (remaining_boxes[:, 3] - remaining_boxes[:, 1] + 1)
        union = area1 + area2 - intersection
        iou = intersection / union
        
        # Berechnen des Verhältnisses der Überlappung zur Fläche der kleineren Box
        # Dies hilft zu erkennen, ob eine Box fast vollständig in einer anderen enthalten ist
        containment_ratio1 = intersection / area1  # Wie viel von Box 1 ist in der Überlappung enthalten
        containment_ratio2 = intersection / area2  # Wie viel von Box 2 ist in der Überlappung enthalten
        
        # Indizes der Boxen, die entfernt werden sollen
        to_remove = []
        
        for i in range(len(remaining_boxes)):
            # Wenn die IoU über dem Schwellenwert liegt, entfernen wir die Box
            if iou[i] > overlap_threshold:
                to_remove.append(i)
            # Wenn eine Box fast vollständig in der aktuellen Box enthalten ist
            elif containment_ratio2[i] > containment_threshold:
                # Wenn die kleinere Box eine höhere Konfidenz hat, behalten wir sie
                if remaining_boxes[i, 4] > current_box[4] * 1.2:  # 20% höhere Konfidenz
                    continue
                to_remove.append(i)
        
        # Erstellen einer Maske für die zu behaltenden Boxen
        mask = np.ones(len(remaining_boxes), dtype=bool)
        mask[to_remove] = False
        
        # Aktualisieren der verbleibenden Boxen
        boxes_array = remaining_boxes[mask]
    
    # Konvertieren zurück in das Format (x, y, w, h, confidence, object_type)
    picked = [(box[0], box[1], box[2] - box[0], box[3] - box[1], box[4], box[5]) for box in picked]
    
    return picked

### Zeichnen der Bounding Boxes

In [ ]:
def draw_boxes(image, boxes):
    """
    Zeichnet Bounding Boxes auf ein Bild.
    
    Args:
        image: Eingabebild
        boxes: Liste der Bounding Boxes (x, y, w, h, confidence, object_type)
        
    Returns:
        result: Bild mit Bounding Boxes
    """
    # Convert numpy array to PIL Image
    pil_image = Image.fromarray(image)
    draw = ImageDraw.Draw(pil_image)
    
    for (x, y, w, h, conf, obj_type) in boxes:
        # Convert coordinates to integers
        x, y, w, h = int(x), int(y), int(w), int(h)
        
        # Farbe basierend auf Objekttyp
        color = (0, 255, 0) if obj_type == 'car' else (255, 0, 0)  # Grün für Autos, Rot für Personen
        
        # Zeichnen der Bounding Box
        draw.rectangle([(x, y), (x + w, y + h)], outline=color, width=2)
        
        # Zeichnen des Labels
        label = f"{obj_type.capitalize()}: {conf:.2f}"
        draw.text((x, y - 10), label, fill=color)
    
    # Convert back to numpy array
    result = np.array(pil_image)
    
    return result

### Hauptfunktion zur Erkennung von Personen und Autos in Bildern

Diese Funktion kombiniert alle vorherigen Funktionen, um Personen und Autos in einem Bild zu erkennen und zu markieren.

In [ ]:
def detect_and_draw_objects(image_path, person_model, car_model, output_path):
    """
    Erkennt Personen und Autos in einem Bild und zeichnet Bounding Boxes.
    
    Args:
        image_path: Pfad zum Bild oder URL
        person_model: Trainiertes Modell für Personenerkennung
        car_model: Trainiertes Modell für Autoerkennung
        output_path: Pfad zum Ausgabebild
        
    Returns:
        boxes: Liste der erkannten Objekte (x, y, w, h, confidence, object_type)
    """
    # Laden und Vorverarbeiten des Bildes
    image, processed_image, (original_height, original_width) = load_and_preprocess_image(image_path)
    
    # Zeitmessung starten
    start_time = time.time()
    
    # Erkennen von Personen im Bild
    person_boxes = detect_objects_multi_scale(image, person_model, 'person', confidence_threshold=0.6)
    
    # Erkennen von großen Personen
    large_person_boxes = detect_large_objects(image, person_model, 'person', confidence_threshold=0.6)
    person_boxes.extend(large_person_boxes)
    
    # Erkennen von Autos im Bild
    car_boxes = detect_objects_multi_scale(image, car_model, 'car', confidence_threshold=0.6)
    
    # Erkennen von großen Autos
    large_car_boxes = detect_large_objects(image, car_model, 'car', confidence_threshold=0.6)
    car_boxes.extend(large_car_boxes)
    
    # Kombinieren der Erkennungen
    all_boxes = person_boxes + car_boxes
    
    # Zusammenführen überlappender Bounding Boxes mit erweiterter NMS
    boxes = advanced_non_max_suppression(all_boxes, overlap_threshold=0.3, score_threshold=0.6, containment_threshold=0.95)
    
    # Zeitmessung beenden
    end_time = time.time()
    processing_time = end_time - start_time
    
    # Zeichnen der Bounding Boxes
    result = draw_boxes(image, boxes)
    
    # Hinzufügen von Informationen zum Bild
    pil_result = Image.fromarray(result)
    draw = ImageDraw.Draw(pil_result)
    person_count = sum(1 for box in boxes if box[5] == 'person')
    car_count = sum(1 for box in boxes if box[5] == 'car')
    info_text = f"Erkannte Personen: {person_count} | Erkannte Autos: {car_count} | Verarbeitungszeit: {processing_time:.2f}s"
    draw.text((10, 30), info_text, fill=(0, 0, 255))
    result = np.array(pil_result)
    
    # Speichern des Ergebnisses
    Image.fromarray(result).save(output_path)
    
    # Erstellen einzelner Bilder für jedes erkannte Objekt
    for i, (x, y, w, h, conf, obj_type) in enumerate(boxes):
        # Convert coordinates to integers
        x, y, w, h = int(x), int(y), int(w), int(h)
        
        # Extrahieren des Objekts
        object_image = image[y:y+h, x:x+w]
        
        # Zeichnen einer Box um das Objekt
        pil_object = Image.fromarray(object_image)
        draw = ImageDraw.Draw(pil_object)
        color = (0, 255, 0) if obj_type == 'car' else (255, 0, 0)  # Grün für Autos, Rot für Personen
        draw.rectangle([(0, 0), (w, h)], outline=color, width=2)
        object_image_with_box = np.array(pil_object)
        
        # Speichern des Bildes
        object_output_path = output_path.replace('.jpg', f'_{obj_type}_{i+1}.jpg')
        Image.fromarray(object_image_with_box).save(object_output_path)
    
    return boxes

## Anwendung auf Testbilder

### Testen der Personen- und Autoerkennung auf Bildern

In [ ]:
print("Testen der Personen- und Autoerkennung auf Bildern...")

# Testbilder mit Personen und Autos
test_images = [
    "https://cdn.pixabay.com/photo/2017/08/06/15/13/people-2593378_1280.jpg",  # Personen auf der Straße
    "https://cdn.pixabay.com/photo/2016/11/18/12/14/car-1834274_1280.jpg",     # Auto mit Person
    "https://cdn.pixabay.com/photo/2017/08/01/11/48/woman-2564660_1280.jpg"    # Person neben Auto
]

for i, image_url in enumerate(test_images):
    # Speichern des Bildes
    image_path = os.path.join(images_dir, f'test_image_{i+1}.jpg')
    
    # Herunterladen des Bildes, wenn es noch nicht existiert
    if not os.path.exists(image_path):
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)
    
    # Erkennen von Personen und Autos im Bild
    output_path = os.path.join(bonus_dir, f'test_image_{i+1}_result.jpg')
    boxes = detect_and_draw_objects(image_path, person_model, car_model, output_path)
    
    # Zählen der erkannten Objekte
    person_count = sum(1 for box in boxes if box[5] == 'person')
    car_count = sum(1 for box in boxes if box[5] == 'car')
    
    print(f"Bild {i+1}: {person_count} Personen und {car_count} Autos erkannt")
    
    # Anzeigen des Ergebnisses im Notebook
    plt.figure(figsize=(12, 8))
    plt.imshow(plt.imread(output_path))
    plt.title(f"Bild {i+1}: {person_count} Personen und {car_count} Autos erkannt")
    plt.axis('off')
    plt.show()

### Testen der Personen- und Autoerkennung auf den Bildern aus dem Repository

In [ ]:
print("Testen der Personen- und Autoerkennung auf den Bildern aus dem Repository...")

# Bilder aus dem Repository
repo_images = glob.glob(os.path.join(images_dir, 'bild*.jpg'))

for i, image_path in enumerate(repo_images):
    # Erkennen von Personen und Autos im Bild
    output_path = os.path.join(bonus_dir, f'repo_image_{i+1}_result.jpg')
    boxes = detect_and_draw_objects(image_path, person_model, car_model, output_path)
    
    # Zählen der erkannten Objekte
    person_count = sum(1 for box in boxes if box[5] == 'person')
    car_count = sum(1 for box in boxes if box[5] == 'car')
    
    print(f"Repository-Bild {i+1}: {person_count} Personen und {car_count} Autos erkannt")
    
    # Anzeigen des Ergebnisses im Notebook
    plt.figure(figsize=(12, 8))
    plt.imshow(plt.imread(output_path))
    plt.title(f"Repository-Bild {i+1}: {person_count} Personen und {car_count} Autos erkannt")
    plt.axis('off')
    plt.show()

## Zusammenfassung

In diesem Notebook haben wir:
1. Einen Datensatz für Personenerkennung aus dem HumanBinaryClassificationSuite Repository geladen und vorbereitet
2. Ein CNN-Modell für Personenerkennung definiert und trainiert
3. Das trainierte Personenerkennungsmodell und das vorhandene Autoerkennungsmodell geladen
4. Eine verbesserte Erkennungsmethode basierend auf dem 05-Notebook implementiert, die Region Proposals und Multi-Scale-Erkennung verwendet
5. Eine spezielle Erkennung für große Objekte implementiert, die einen signifikanten Teil des Bildes einnehmen
6. Eine erweiterte Non-Maximum Suppression implementiert, die 100% überlappende Boxen entfernt und eine intelligente Filterung für verschachtelte Boxen bietet
7. Die Erkennung auf verschiedenen Testbildern angewendet und die Ergebnisse visualisiert

Der implementierte Ansatz ermöglicht eine zuverlässige Erkennung von Personen und Autos in Bildern unterschiedlicher Größen und Perspektiven. Die Verwendung von Region Proposals anstelle des Sliding-Window-Ansatzes führt zu einer besseren Erkennung, insbesondere in komplexen Szenen mit mehreren Objekten. Die spezielle Erkennung für große Objekte ermöglicht es, Personen und Autos zu erkennen, die einen signifikanten Teil des Bildes einnehmen. Die erweiterte Non-Maximum Suppression verhindert 100% überlappende Boxen und bietet eine intelligente Filterung für verschachtelte Boxen, die nicht einfach kleinere Boxen innerhalb größerer Boxen entfernt, sondern die Konfidenz und das Enthaltensein berücksichtigt.

In [ ]:
print("Personen- und Autoerkennung abgeschlossen. Die Ergebnisse wurden im Verzeichnis 'bonus' gespeichert.")